Neural Point Process

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from skimage import io, transform
from functools import lru_cache
from tools.plot_utils import visualize_pins, plot_label_pin, plot_all, plot_and_save, plot_loss
from tools.data_utils import *
from tools.losses import NPPLoss
from tools.models import Autoencoder
from tools.optimization import EarlyStoppingCallback, train_model, evaluate_model
import matplotlib.pyplot as plt
# from torch_lr_finder import LRFinder
import time
from tools.models import *
from torch.utils.data import Subset
import os

In [2]:
torch.__version__

'2.2.1+cu121'

# Dataset and Visualization

In [5]:
# Set a random seed for PyTorch
seed = 4  # You can use any integer value as the seed
torch.manual_seed(seed)
# Set a random seed for NumPy (if you're using NumPy operations)
np.random.seed(seed)

In [6]:
dataset = "PinMNIST"
feature_extracted = False
learn_kernel = False
n = 10
mesh = False
d = 10
n_pins = 100
fixed_pins = True
r = 3
d1, d2 = 28, 28

partial_label_GP = False
partial_percent = 0.5

if feature_extracted:
    folder = f"{dataset}_ddpm"
else:
    folder = f"{dataset}"

if dataset == "PinMNIST":
    if mesh:
        data_folder = f"./data/{folder}/mesh_{d}step"
        config['n_pins'] = (28 // d + 1) ** 2
    else:
        data_folder = f"./data/{folder}/random_{n_pins}pins"
elif dataset == "Synthetic":
    folder += "/28by28pixels_1000images_123456seed"
    if mesh:
        data_folder = f"./data/{folder}/mesh_{d}step_pins"
        config['n_pins'] = (28 // d + 1) ** 2
    else:
        data_folder = f"./data/{folder}/random_{n_pins}pins"
elif dataset == "Building":
    if mesh:
        data_folder = f"./data/{folder}/mesh_{d}_step"
    else:
        data_folder = f"./data/{folder}/random_n_pins_{n_pins}"


In [7]:
if dataset == "Building":
    resize = Resize100
else:
    resize = Resize
        

# Define batch size
batch_size = 16

def custom_collate_fn(batch):
    images = [sample['image'] for sample in batch]
    pins = [sample['pins'] for sample in batch]
    outputs = [sample['outputs'] for sample in batch]

    return {
        'image': torch.stack(images, dim=0),
        'pins': pins,
        'outputs': outputs}

transform = transforms.Compose([
        ToTensor(),  # Convert to tensor (as you were doing)
        resize()  # Resize to 100x100
    ])


modality = "PS-RGBNIR-SAR"
if dataset == "Building":
    root_dir = "/work/USACE_KRI/Project_1/spacenet/aoi_11_rotterdam/train/train/AOI_11_Rotterdam/"+modality+"/"
else:
    root_dir=f"./data/{folder}/images/"
    
transformed_dataset = PinDataset(csv_file=f"{data_folder}/pins.csv",
                                 root_dir=root_dir, modality=modality,
                                 transform=transform)


dataset_size = len(transformed_dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.10 * dataset_size)
test_size = dataset_size - train_size - val_size


if os.path.exists(f"./data/{dataset}/train_indices.npy"):
    train_indices = np.load(f'./data/{dataset}/train_indices.npy')
    val_indices = np.load(f'./data/{dataset}/val_indices.npy')
    test_indices = np.load(f'./data/{dataset}/test_indices.npy')
    # Use the indices to create new datasets
    train_dataset = Subset(transformed_dataset, train_indices)
    val_dataset = Subset(transformed_dataset, val_indices)
    test_dataset = Subset(transformed_dataset, test_indices)
else:
    # Split the dataset into train, validation, and test sets
    train_dataset, val_dataset, test_dataset = random_split(
        transformed_dataset, [train_size, val_size, test_size]
    )
    np.save(f'./data/{dataset}/train_indices.npy', train_dataset.indices)
    np.save(f'./data/{dataset}/val_indices.npy', val_dataset.indices)
    np.save(f'./data/{dataset}/test_indices.npy', test_dataset.indices)

# Create your DataLoader with the custom_collate_fn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for batch in train_loader:
    images = batch['image'].to(device) # get RGB instead of RGBA
    pins = batch['pins']
    outputs = batch['outputs']
    break

In [9]:
images.shape

torch.Size([16, 1, 28, 28])

In [10]:
j=6
sample_img = images[j].squeeze().detach().cpu()
count_image = plot_label_pin(sample_img[0], pins[j], outputs[j])
count_all_image = plot_all(sample_img[0], r=3)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sample_img = sample_img[4:]
img = np.transpose(sample_img, (1, 2, 0))
print(sample_img.shape, img.shape)
im0 = axes[0].imshow(img)
# Plot the sample_img in the first subplot
# im0 = axes[0].imshow(sample_img)
axes[0].set_title('Sample Image')
fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)  # Add colorbar

# Plot the count_image in the second subplot
im1 = axes[1].imshow(count_image)
axes[1].set_title('Count Image')
fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)  # Add colorbar

# Plot the count_all_image in the third subplot
im2 = axes[2].imshow(count_all_image)
axes[2].set_title('Count All Image')
fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)  # Add colorbar

# Add spacing between subplots
plt.tight_layout()
# Display the figure
plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# Training

In [10]:
class CustomLRFinder:
    def __init__(self, model, criterion, optimizer, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.history = {'lr': [], 'loss': []}

    def find_lr(self, train_loader, start_lr=1e-4, end_lr=0.1, num_iter=20,smooth_f=0.05):
        model = self.model.to(self.device)
        criterion = self.criterion
        optimizer = self.optimizer
        device = self.device
        model.train()

        lr_step = (end_lr / start_lr) ** (1 / num_iter)
        lr = start_lr

        for iteration in range(num_iter):
            optimizer.param_groups[0]['lr'] = lr

            total_loss = 0.0
            for batch in train_loader:
                x_train = batch['image'][:, :input_channel, :, :].to(device)
                p_train = [tensor.to(device) for tensor in batch['pins']]
                y_train = [tensor.to(device) for tensor in batch['outputs']]

                optimizer.zero_grad()
                outputs = model(x_train.float())
                loss = criterion(y_train, outputs, p_train)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            self.history['lr'].append(lr)
            self.history['loss'].append(avg_loss)

            lr *= lr_step
            
    def plot_lr_finder(self):
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')  # Use a logarithmic scale for better visualization
        plt.xlabel('Learning Rate')
        plt.ylabel('Loss')
        plt.title('Learning Rate Finder Curve')
        plt.show()
        
    def find_best_lr(self, skip_start=3, skip_end=3):
        # Find the index of the minimum loss in the specified range
        min_loss_index = skip_start + np.argmin(self.history['loss'][skip_start:-skip_end])

        # Output the learning rate corresponding to the minimum loss
        best_lr = self.history['lr'][min_loss_index]
        return best_lr

In [11]:
def run_pipeline_ci(sigmas, num_kernels_encoder, num_kernels_decoder, train_loader, val_loader,
                    test_loader, input_channel, epochs, val_every_epoch, config, device, num_runs=3, exp_name=""):
    GP_test_losses_npp_true = []
    test_losses_npp_true = []
    test_losses_npp_false = []
    GP_r2_losses_npp_true = []
    r2_losses_npp_true = []
    r2_losses_npp_false = []
    partial_percent = config['partial_percent']
    experiment_id = int(time.time())
    best_val_loss_MSE = float('inf')
    best_val_loss_NPP = float('inf')
    best_sigma_NPP = float('inf')
    config['experiment_id'] = experiment_id
    deeper = config['deeper']
    manual_lr = config['manual_lr']
    learning_rates = config['best_lrs']
    losses = {}

    # Create storage directory and store the experiment configuration
    if not os.path.exists(f'./history/{exp_name}/{experiment_id}'):
        os.makedirs(f'./history/{exp_name}/{experiment_id}')
    with open(f"./history/{exp_name}/{experiment_id}/config.json", "w") as outfile:
        json.dump(config, outfile)

    for run in range(num_runs):
        count = 0
        test_losses_vs_sigma_npp_true = []
        R2_losses_vs_sigma_npp_true = []
        GP_test_losses_vs_sigma_npp_true = []
        GP_R2_losses_vs_sigma_npp_true = []
        for sigma in sigmas:
            early_stopping = EarlyStoppingCallback(patience=15, min_delta=0.001)
            autoencoder = Autoencoder(num_kernels_encoder, num_kernels_decoder, input_channel=input_channel, deeper=deeper).to(device)
            lr = learning_rates[count][1]
            optimizer = optim.Adam(autoencoder.parameters(), lr=lr)
            print(f"training start for sigma:{sigma}")
            if sigma == 0:
                # run plain
                
                criterion = NPPLoss(identity=True).to(device)
                model, train_losses, val_losses, best_val_loss = train_model(autoencoder, train_loader, val_loader,
                                                                             input_channel, epochs, \
                                                                             val_every_epoch, lr,
                                                                             criterion, optimizer, device, early_stopping,
                                                                             experiment_id, exp_name, best_val_loss_MSE, manual_lr, sigma=0)
                losses[f"MSE_run{run}_train"] = train_losses
                losses[f"MSE_run{run}_val"] = val_losses
                if best_val_loss < best_val_loss_MSE:
                    best_val_loss_MSE = best_val_loss

                test_loss_npp_false, r2_loss_npp_false = evaluate_model(autoencoder, test_loader, input_channel, device,
                                                                        partial_label_GP=False, partial_percent=partial_percent)
                print(f"MSE Test loss:{test_loss_npp_false:.3f}")
                print(f"R2 Test loss:{r2_loss_npp_false:.3f}")
                test_losses_npp_false.append(test_loss_npp_false)
                r2_losses_npp_false.append(r2_loss_npp_false)        
            else:
                # run NPP
                criterion = NPPLoss(identity=False, sigma=sigma).to(device)               
                
                model, train_losses, val_losses, best_val_loss = train_model(autoencoder, train_loader, val_loader,
                                                                             input_channel, epochs, \
                                                                             val_every_epoch, lr,
                                                                             criterion, optimizer, device, early_stopping,
                                                                             experiment_id, exp_name, best_val_loss_NPP, manual_lr, sigma=sigma)
                losses[f"NPP_run{run}_sigma{sigma}_train"] = train_losses
                losses[f"NPP_run{run}_sigma{sigma}_val"] = val_losses
                if best_val_loss < best_val_loss_NPP:
                    best_val_loss_NPP = best_val_loss
                    best_sigma_NPP = sigma

                test_loss, r2_loss = evaluate_model(autoencoder, test_loader, input_channel, device,
                                                    partial_label_GP=False, partial_percent=partial_percent)
                GP_test_loss, GP_r2_loss = evaluate_model(autoencoder, test_loader, input_channel, device,
                                                          partial_label_GP=True, partial_percent=partial_percent)
                print(f"NPP sigma={sigma} Test loss:{test_loss:.3f}, R2 loss:{r2_loss:.3f}, GP Test loss:{GP_test_loss:.3f}"
                      f", GP R2 loss:{GP_r2_loss:.3f}")
                test_losses_vs_sigma_npp_true.append(test_loss)
                R2_losses_vs_sigma_npp_true.append(r2_loss)
                GP_test_losses_vs_sigma_npp_true.append(GP_test_loss)
                GP_R2_losses_vs_sigma_npp_true.append(GP_r2_loss)
            count += 1

        test_losses_npp_true.append(test_losses_vs_sigma_npp_true)
        GP_test_losses_npp_true.append(GP_test_losses_vs_sigma_npp_true)
        r2_losses_npp_true.append(R2_losses_vs_sigma_npp_true)
        GP_r2_losses_npp_true.append(GP_R2_losses_vs_sigma_npp_true)
    with open(f"./history/{exp_name}/{experiment_id}/losses.json", "w") as outfile:
        json.dump(losses, outfile)
    return GP_test_losses_npp_true, test_losses_npp_true, test_losses_npp_false, r2_losses_npp_false, r2_losses_npp_true, GP_r2_losses_npp_true, best_sigma_NPP, experiment_id

    
# Function to run the pipeline and save data
def run_and_save_pipeline(sigmas, num_kernels_encoder, num_kernels_decoder, train_loader, val_loader, test_loader,
                          input_channel, epochs, val_every_epoch, config, num_runs, exp_name, device):
    # Run the pipeline
    GP_test_loss_npp_true, test_loss_npp_true, test_loss_npp_false, r2_loss_npp_false, r2_losses_npp_true, GP_r2_losses_npp_true, best_sigma_NPP, experiment_id = run_pipeline_ci(
        sigmas, num_kernels_encoder, num_kernels_decoder, train_loader, val_loader, test_loader, input_channel, epochs,
        val_every_epoch, config, device, num_runs, exp_name)
    partial_percent = config['partial_percent']
    # Run final testing
    autoencoder = Autoencoder(num_kernels_encoder, num_kernels_decoder, input_channel=input_channel, deeper=config["deeper"]).to(device)
    if sigmas[0] == 0:
        # MSE
        autoencoder.load_state_dict(torch.load(f'./history/{exp_name}/{experiment_id}/best_model_MSE.pth'))
        best_MSE_test_loss, best_R2_test_loss = evaluate_model(autoencoder, test_loader, input_channel, device,
                                                               partial_label_GP=False,
                                                               partial_percent=partial_percent)
        f = open(f"./history/{exp_name}/{experiment_id}/results.txt", "w")
        f.write(f"Results {experiment_id}:\n MSE: {best_MSE_test_loss}, R2: {best_R2_test_loss} ")
        f.close()
        print("metrics saved")
    else:
        # NPP
        autoencoder.load_state_dict(torch.load(f'./history/{exp_name}/{experiment_id}/best_model_NPP.pth'))
        best_NPP_test_loss, best_NPP_R2_test_loss = evaluate_model(autoencoder, test_loader, input_channel, device,
                                                                   partial_label_GP=False,
                                                                   partial_percent=partial_percent)
        GP_best_NPP_test_loss, GP_best_NPP_r2_test_loss = evaluate_model(autoencoder, test_loader, input_channel, device,
                                                                         partial_label_GP=True,
                                                                         partial_percent=partial_percent)
        f = open(f"./history/{exp_name}/{experiment_id}/results.txt", "w")
        f.write(f"| NPP (sigma {best_sigma_NPP}): {best_NPP_test_loss}, R2: {best_NPP_R2_test_loss}; GP: {GP_best_NPP_test_loss}, R2: {GP_best_NPP_r2_test_loss}")
        f.close()
        print("metrics saved")
        
    print("start saving losses!")
    # Save losses
    save_loss(test_loss_npp_true, f'./history/{exp_name}/{experiment_id}/test_loss_npp_true.npy')
    save_loss(test_loss_npp_false, f'./history/{exp_name}/{experiment_id}/test_loss_npp_false.npy')
    save_loss(GP_test_loss_npp_true, f'./history/{exp_name}/{experiment_id}/GP_test_loss_npp_true.npy')
    # Save r2 scores
    save_loss(r2_loss_npp_false, f'./history/{exp_name}/{experiment_id}/r2_loss_npp_false.npy')
    save_loss(r2_losses_npp_true, f'./history/{exp_name}/{experiment_id}/r2_losses_npp_true.npy')
    save_loss(GP_r2_losses_npp_true, f'./history/{exp_name}/{experiment_id}/GP_r2_losses_npp_true.npy')
    
    return (GP_test_loss_npp_true, test_loss_npp_true, test_loss_npp_false), (r2_loss_npp_false, r2_losses_npp_true, GP_r2_losses_npp_true), experiment_id

In [13]:
# Generate a unique experiment_id using a timestamp
  # Using timestamp as experiment_id

# Set your hyperparameters
# dataset = "MNIST"
input_channel = 1 if dataset == "PinMNIST" else 3
epochs = 200
sigmas = [0.1, 0.2, 0.5, 1, 2]  # Set the sigma values you want to test
# best_lrs = [0.05, 0.05, 0.05, 0.05, 0.05, 0.001] #MNIST
best_lrs = [0.1, 0.001, 0.001, 0.001, 0.001, 0.001] #Synthetic
num_kernels_encoder = [64, 32]
num_kernels_decoder = [64]
# learning_rate = 0.01
val_every_epoch = 5
num_runs = 1
learn_kernel = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = {
    "dataset": "PinMNIST",
    "modality": "PS-RGBNIR",
    "feature": "AE",
    "mode": "mesh",
    "n": 100,
    "d": 10,
    "n_pins": 500,
    "partial_percent": 0.00,
    "r": 3,
    "epochs": 200,
    "batch_size": 64,
    "learning_rate": 0.1,
    "val_every_epoch": 5,
    "num_runs": 3,
    "manual_lr": False,
    "sigmas": [0],
    "num_encoder": [64, 32],
    "num_decoder": [64],
    "deeper": False,
    "experiment_name": None
}

# Run and save the pipeline data
loss_vs_sigma_data = run_and_save_pipeline(sigmas, num_kernels_encoder, num_kernels_decoder, train_loader, val_loader, test_loader,
                          input_channel, epochs, val_every_epoch, config, num_runs, exp_name, device)

# Plot and save the plot using the saved data
plot_and_save(loss_vs_sigma_data, sigmas, dataset, learning_rate)

TypeError: list indices must be integers or slices, not str

In [ ]:
device

In [ ]:
loss_vs_sigma_data

In [ ]:
# test_loss_npp_false = [test_loss_npp_false for i in range((len(sigmas)))]
# test_loss_npp_true.pop(1)
test_loss_npp_true, test_loss_npp_false = load_data('./history/test_loss_npp_true.npy'), load_data('./history/test_loss_npp_false.npy')
test_loss_npp_true, test_loss_npp_false

In [ ]:
def plot_and_save(loss_vs_sigma_data, sigmas, dataset, learning_rate, dataset, model_name="Auto encoder"):
    # Unpack the data
    test_loss_npp_true, test_loss_npp_false = loss_vs_sigma_data
    test_loss_npp_false = [test_loss_npp_false for i in range(len(sigmas))]

    # Calculate mean and confidence intervals for NPP=True runs
    mean_test_loss_npp_true = np.mean(test_loss_npp_true, axis=0)
    ci_test_loss_npp_true = 1.96 * np.std(test_loss_npp_true, axis=0) / np.sqrt(len(test_loss_npp_true))

    # Duplicate NPP=False values for plotting
    mean_test_loss_npp_false = np.mean(test_loss_npp_false, axis=1)
    ci_test_loss_npp_false = 1.96 * np.std(test_loss_npp_false, axis=1) / np.sqrt(len(test_loss_npp_false))

    # Plot mean and confidence intervals for NPP=True
    plt.plot(sigmas, mean_test_loss_npp_true, marker='o', label='NPP=True', color='blue')

    # Plot mean and confidence intervals for duplicated NPP=False
    plt.plot(sigmas, mean_test_loss_npp_false, color='red', linestyle='--', label='NPP=False')

    # Fill between for NPP=True with blue color
    plt.fill_between(sigmas, mean_test_loss_npp_true - ci_test_loss_npp_true, mean_test_loss_npp_true + ci_test_loss_npp_true, color='blue', alpha=0.2)

    # Fill between for NPP=False with red color
    plt.fill_between(sigmas, mean_test_loss_npp_false - ci_test_loss_npp_false, mean_test_loss_npp_false + ci_test_loss_npp_false, color='red', alpha=0.2)

    plt.xlabel('Sigma')
    plt.ylabel('Test Loss')
    plt.title(f'Test Loss vs. Sigma:{dataset} dataset with {model_name}')
    plt.legend()

    # Create a directory to save the results if it doesn't exist
    results_dir = './results'
    os.makedirs(results_dir, exist_ok=True)

    # Generate a filename based on parameters in the title
    filename = f"test_loss_vs_sigma_{dataset}_{model_name}_lr_{learning_rate}.png"
    filepath = os.path.join(results_dir, filename)

    # Save the plot
    plt.savefig(filepath)

    # Show the plot
    plt.show()

# Example usage:
# plot_and_save(loss_vs_sigma_data, sigmas, dataset, learning_rate)

# Plot and save the plot using the saved data
plot_and_save(loss_vs_sigma_data, sigmas, dataset, learning_rate)